In [ ]:
! pip install -Uqqq pip --progress-bar off
! pip install -qqq bitsandbytes==0.39.0 --progress-bar off
! pip install -qqq torch==2.0.1 --progress-bar off
! pip install -qqq transformers==4.30.0 --progress-bar off
! pip install -qqq accelerate==0.20.3 --progress-bar off
! pip install -qqq -U git+https://github.com/huggingface/peft.git --progress-bar off
! pip install -qqq datasets==2.12.0 --progress-bar off
! pip install -qqq loralib==0.1.1 --progress-bar off
! pip install -qqq einops==0.6.1 --progress-bar off

In [ ]:

import torch
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [ ]:
#falcon7b with training config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model_id = "vilsonrodrigues/falcon-7b-instruct-sharded"
model_4bit  = AutoModelForCausalLM.from_pretrained(
    model_id,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

generation_config = model_4bit.generation_config
generation_config.max_new_tokens = 100
generation_config.temperature = 0
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

DEVICE = 'cuda:3'

prompt = "identify the spelling errors in the following text. 'wht are you dong?'// 2 'what' and 'doing'.  'hw you seing' -- "
encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)


In [ ]:
%%time

with torch.inference_mode():
    outputs = model_4bit.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
        do_sample=False,
        use_cache=True,
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
--END--

In [ ]:
#falcon7b 4 bits
model_id = "vilsonrodrigues/falcon-7b-instruct-sharded"
model_4bit  = AutoModelForCausalLM.from_pretrained(
    model_id,
    return_dict=True,
    device_map="auto",
    trust_remote_code=True,
    load_in_4bit=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

generation_config = model_4bit.generation_config
generation_config.max_new_tokens = 20
generation_config.temperature = 0
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

DEVICE = 'cuda:3'

prompt = "identify the spelling errors in the following text. 'wht are you dong?'"
encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)

In [ ]:
%%time

with torch.inference_mode():
    outputs = model_4bit.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
        do_sample=False,
        use_cache=True,
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
#falcon7b 8 bits
model_id = "vilsonrodrigues/falcon-7b-instruct-sharded"
model_8bit  = AutoModelForCausalLM.from_pretrained(
    model_id,
    return_dict=True,
    device_map="auto",
    trust_remote_code=True,
    load_in_8bit=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

generation_config = model_8bit.generation_config
generation_config.max_new_tokens = 20
generation_config.temperature = 0
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

DEVICE = 'cuda:3'

prompt = "identify the spelling errors in the following text. 'wht are you dong?'"
encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)

In [ ]:
%%time

with torch.inference_mode():
    outputs = model_8bit.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
        do_sample=False,
        use_cache=True,
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
#falcon7b with training config 8 bit and float16
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_use_double_quant=True,
    bnb_8bit_quant_type="nf4",
    bnb_8bit_compute_dtype=torch.bfloat16,
)

model_id = "vilsonrodrigues/falcon-7b-instruct-sharded"
model_8bit  = AutoModelForCausalLM.from_pretrained(
    model_id,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

generation_config = model_8bit.generation_config
generation_config.max_new_tokens = 20
generation_config.temperature = 0
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

DEVICE = 'cuda:3'

prompt = "identify the spelling errors in the following text. 'wht are you dong?'"
encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)

In [ ]:
%%time

with torch.inference_mode():
    outputs = model_8bit.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
        do_sample=False,
        use_cache=True,
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b", padding_side="left")
tokenizer.pad_token = tokenizer.eos_token

DEVICE = "cuda:2"

prompt = f"""
: count the number of spelling mistakes in the following text? "hi, hw are yu doing?"
:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [ ]:
MODEL_ID = "curiousily/falcon-7b-qlora-chat-support-bot-faq"
config = PeftConfig.from_pretrained(MODEL_ID)

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

model = PeftModel.from_pretrained(model, MODEL_ID)

generation_config = model.generation_config
generation_config.max_new_tokens = 20
generation_config.temperature = 0
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
        do_sample=False,
        use_cache=True,
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))
     

In [ ]:
## 4-bit
MODEL_ID = "curiousily/falcon-7b-qlora-chat-support-bot-faq"
config = PeftConfig.from_pretrained(MODEL_ID)

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    device_map="auto",
    trust_remote_code=True,
    load_in_4bit=True,
)

model = PeftModel.from_pretrained(model, MODEL_ID)

generation_config = model.generation_config
generation_config.max_new_tokens = 20
generation_config.temperature = 0
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
        do_sample=False,
        use_cache=True,
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
##8 bit

MODEL_ID = "curiousily/falcon-7b-qlora-chat-support-bot-faq"
config = PeftConfig.from_pretrained(MODEL_ID)

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    device_map="auto",
    trust_remote_code=True,
    load_in_8bit=True,
)

model = PeftModel.from_pretrained(model, MODEL_ID)

generation_config = model.generation_config
generation_config.max_new_tokens = 20
generation_config.temperature = 0
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id



with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
        do_sample=False,
        use_cache=True,
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
model = torch.compile(model)


with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
        do_sample=False,
        use_cache=True,
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
import torch
torch.cuda.empty_cache()